# COMP7015: Artificial Intelligence *(Semester 1, 2023/24)*

# Lab1: Solving Problems by Searching

In this lab session, we will learn how to solve problems using search algorithms. Specifically, we will learn the implementation of BFS, DFS, uniform-cost search and greedy search using the path-finding problem. The A* search will be left to you as an exercise.


**Course Instructor: Dr. Kejing Yin (Department of Computer Science, Hong Kong Baptist University)**  
**Lab Instructor: Mr. Kenny Cheng (Department of Computer Science, Hong Kong Baptist University)**

*This lab sheet is created by Dr. Kejing Yin and is licenced under the MIT license.*

> MIT License
> 
> Copyright (c) 2023 Kejing Yin
> 
> Permission is hereby granted, free of charge, to any person obtaining a copy
> of this software and associated documentation files (the "Software"), to deal
> in the Software without restriction, including without limitation the rights
> to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
> copies of the Software, and to permit persons to whom the Software is
> furnished to do so, subject to the following conditions:
> 
> The above copyright notice and this permission notice shall be included in all
> copies or substantial portions of the Software.
> 
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
> IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
> FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
> AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
> LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
> OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
> SOFTWARE.

## 1. Queues for the frontiers list

<div>
    <img src="figs/fifo_lifo.png"/>
</div>


**FIFO Queue**: the last element inserted/recorded is consumed first.

**LIFO Queue**: the oldest element is consumed first.

**Priority Queue**: each element is associated with a priority value and the elements with minimal values are consumed first.

In Python, we can use the standard library [queue](https://docs.python.org/3/library/queue.html). 

Please do read its documentations: https://docs.python.org/3/library/queue.html

Let's see an example:

In [1]:
# import the package
import queue

# Define a FIFO queue
frontiers = queue.Queue()

# use `qsize` method to get the approximate size of the queue
print('Size of the queue before adding any elements into it:', frontiers.qsize())

# use `empty` method to check if it is empty
print('Is the queue empty?', frontiers.empty())

print()
for x in ['A', 'B', 'C']:
    print(f'Putting {x} into the queue')
    frontiers.put(x)  # put x into the queue
    print('Size of the queue:', frontiers.qsize())  # check the size
    
print()
print('Is the queue empty now?', frontiers.empty())

Size of the queue before adding any elements into it: 0
Is the queue empty? True

Putting A into the queue
Size of the queue: 1
Putting B into the queue
Size of the queue: 2
Putting C into the queue
Size of the queue: 3

Is the queue empty now? False


To get an element, we use the `get` method.

In [2]:
while not frontiers.empty():
    print('The next value we get is:', frontiers.get())

The next value we get is: A
The next value we get is: B
The next value we get is: C


We are using FIFO queue. You can see that the first element is A, which is the first element we put into the queue.

### LIFO Queue

In [3]:
# define a LIFO queue
frontiers = queue.LifoQueue()

In [4]:
# again, we add three elements into the queue
for x in ['A', 'B', 'C']:
    print(f'Putting {x} into the queue')
    frontiers.put(x)  # put x into the queue
    print('Size of the queue:', frontiers.qsize())  # check the size

Putting A into the queue
Size of the queue: 1
Putting B into the queue
Size of the queue: 2
Putting C into the queue
Size of the queue: 3


In [5]:
# take out elements one by one
while not frontiers.empty():
    print('The next value we get is:', frontiers.get())

The next value we get is: C
The next value we get is: B
The next value we get is: A


In FIFO queue, the first element we get is C, which is the last element we put into the queue.

### Priority Queue

In [6]:
# define a Priority queue
frontiers = queue.PriorityQueue()

In [7]:
# again, we add three elements into the queue, but this time, we add together with a priority value.
frontiers.put((2, 'A'))
frontiers.put((1, 'B'))
frontiers.put((5, 'C'))

print('Size of the queue:', frontiers.qsize())  # check the size

Size of the queue: 3


In [8]:
# take out elements one by one
while not frontiers.empty():
    print('The next value we get is:', frontiers.get())

The next value we get is: (1, 'B')
The next value we get is: (2, 'A')
The next value we get is: (5, 'C')


In Priority queue, the first element we get is B, which is the element with the smallest priority value.

## 2. Pseudo-codes of search algorithms

All search algorithms can be summarized using the following pseudo codes.
</br>
</br>
<div>
    <img src="figs/pseudo_code.png" width=95%/>
</div>
The differences between different algorithms is how they choose the node to be expanded next, when to test if the node is a goal state, and whether store the visited nodes.

## 3. How to `expand` a node

For each search problem, we need to know all possible actions and the associated costs from a particular state and the resulting state of taking each action. In the path-finding problem, for each state (location in the map), we can simply store the locations that we can go to from the given location and the cost of taking each action. For example, given the following map:


<div>
    <img src="figs/map1.png" width="400px"/>
</div>

A simple way to represent this map is to use a `dict`.

In [9]:
graph = {
    'A': [('B', 1), ('D', 2)],  # from A, we can go to B and D, with costs 1 and 2, respectively.
    'B': [('A', 1), ('C', 4), ('D', 2)],  # from B, we can go to A, C, and D
    'C': [('B', 4)],  # from C, we can go to B
    'D': [('A', 2), ('B', 2)]  # from D, we can got to A and B
}

In [10]:
# to retrieve the actions, we can use the current state to index this dict
print(graph['B'])  # print out the possible actions from the location B

[('A', 1), ('C', 4), ('D', 2)]


Later, we will see how to define a function to get possible actions from each state for other problems like 8-puzzle.

### *Try it out!*

Represent the following map in a `dict` named "romanian_graph".

<div>
    <img src="figs/romanian_graph.png" width="100%"/>
</div>


In [11]:
romanian_graph = {
    'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],  # finish the remaining
    'Zerind': [('Oradea', 71), ('Arad', 75)],
    'Oradea': [('Zerind', 71), ('Sibiu', 151)],
    'Sibiu': [('Fagaras', 99), ('Rimnicu Vilcea', 80), ('Oradea', 151), ('Arad', 140)],
    'Timisoara': [('Lugoj', 111), ('Arad', 118)],
    'Lugoj': [('Mehadia', 70), ('Timisoara', 111)],
    'Mehadia': [('Drobeta', 75), ('Lugoj', 70)],
    'Drobeta': [('Craiova', 120), ('Mehadia', 75)],
    'Rimnicu Vilcea': [('Craiova', 146), ('Pitesti', 97), ('Sibiu', 80)],
    'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
    'Pitesti': [('Rimnicu Vilcea', 97), ('Craiova', 138), ('Bucharest', 101)],
    'Craiova': [('Rimnicu Vilcea', 146), ('Drobeta', 120), ('Pitesti', 138)],
    'Bucharest': [('Fagaras', 211), ('Pitesti', 101), ('Giurgiu', 90), ('Urzicenzi', 85)],
    'Giurgiu': [('Bucharest', 90)],
    'Urzicenzi': [('Bucharest', 85), ('Vaslui', 142), ('Hirsova', 98)],
    'Vaslui': [('Urzicenzi', 142), ('Iasi', 92)],
    'Iasi': [('Vaslui', 92), ('Neamt', 87)],
    'Neamt': [('Iasi', 87)],
    'Hirsova': [('Urzicenzi', 98), ('Eforie', 86)],
    'Eforie': [('Hirsova', 86)],
}

## 4. Breadth-first Search (BFS)

In [12]:
def breadth_first_search(graph_to_search, initial_state, goal_state, verbose=False):
    # In frontiers, we store a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in the queue represents a path from the the initial
    # state to one frontier node.
    frontiers = queue.Queue()
    frontiers.put((0, [initial_state])) # a tuple of the cost and the path
    visited = set()  # a set

    while not frontiers.empty():   # use while loop to iteratively perform search
        
        cost, path = frontiers.get()  # Get the next element in the frontier queue
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then drop it.
            continue
            
        if verbose:
            print(f'Expanding {node}...')
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            if next_node in visited:
                continue  # skip this node if it is already expanded.
                
            new_path = path.copy()
            new_path.append(next_node)
            new_cost = cost + next_cost
            
            # check if we reache the goal state or not
            if next_node == goal_state:
                return new_path, new_cost  # if yes, we can return this path and its cost
            else:
                frontiers.put((new_cost, new_path))  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited set
        visited.add(node)

    return None, None

In [13]:
graph = {
    'A': [('B', 1), ('D', 2)],
    'B': [('A', 1), ('C', 4), ('D', 2)],
    'C': [('B', 4)],
    'D': [('A', 2), ('B', 2)]
}

path, cost = breadth_first_search(graph, 'A', 'C')
print('The solution is:', path)
print('The cost is:', cost)

The solution is: ['A', 'B', 'C']
The cost is: 5


### *Try it out!*

In [14]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using BFS

path, cost = breadth_first_search(romanian_graph, 'Arad', 'Bucharest', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Zerind...
Expanding Sibiu...
Expanding Timisoara...
Expanding Oradea...
Expanding Fagaras...
The solution is: ['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
The cost is: 450


In [15]:
# Find a path from Arad to Craiova in the Romannian map using BFS

path, cost = breadth_first_search(romanian_graph, 'Arad', 'Craiova', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Zerind...
Expanding Sibiu...
Expanding Timisoara...
Expanding Oradea...
Expanding Fagaras...
Expanding Rimnicu Vilcea...
The solution is: ['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Craiova']
The cost is: 366


In [16]:
# Find a path from Arad to Vaslui in the Romannian map using BFS

path, cost = breadth_first_search(romanian_graph, 'Arad', 'Vaslui', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Zerind...
Expanding Sibiu...
Expanding Timisoara...
Expanding Oradea...
Expanding Fagaras...
Expanding Rimnicu Vilcea...
Expanding Lugoj...
Expanding Bucharest...
Expanding Craiova...
Expanding Pitesti...
Expanding Mehadia...
Expanding Giurgiu...
Expanding Urzicenzi...
The solution is: ['Arad', 'Sibiu', 'Fagaras', 'Bucharest', 'Urzicenzi', 'Vaslui']
The cost is: 677


## 5. Uniform-Cost Search

For uniform-cost search, there are two points that we need to pay attention to:
- First, we check if a node is a goal state **when we expand it**. Think about how to do this in the codes.
- Second, we select the node from frontiers with the **smallest path cost** from the initial state.

A tip: Use priority queue instead of FIFO queue.

### *Try it out!*
Copy the function for bfs, rename it to `uniform_cost_search` and modify it to perform a UCS. Test it with the Romannian map.

In [17]:
def uniform_cost_search(graph_to_search, initial_state, goal_state, verbose=False):
    # In frontiers, we store a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in the queue represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost of the path.
    frontiers = queue.PriorityQueue()
    frontiers.put((0, [initial_state]))
    visited = set()

    while not frontiers.empty():   # use while loop to iteratively perform search
        
        cost, path = frontiers.get()  # Get the next element in the frontier queue
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then drop it.
            continue
        
        if node == goal_state:
            return path, cost
        
        if verbose:
            print(f'Expanding {node}...')
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            if next_node in visited:
                continue  # skip this node if it is already expanded.
                
            new_path = path.copy()
            new_path.append(next_node)
            new_cost = cost + next_cost
            
            frontiers.put((new_cost, new_path))  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited set
        visited.add(node)

    return None, None

In [18]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using BFS

path, cost = uniform_cost_search(romanian_graph, 'Arad', 'Bucharest', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Zerind...
Expanding Timisoara...
Expanding Sibiu...
Expanding Oradea...
Expanding Rimnicu Vilcea...
Expanding Lugoj...
Expanding Fagaras...
Expanding Mehadia...
Expanding Pitesti...
Expanding Craiova...
Expanding Drobeta...
The solution is: ['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']
The cost is: 418


In [19]:
# Find a path from Arad to Vaslui in the Romannian map using BFS

path, cost = uniform_cost_search(romanian_graph, 'Arad', 'Vaslui', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Zerind...
Expanding Timisoara...
Expanding Sibiu...
Expanding Oradea...
Expanding Rimnicu Vilcea...
Expanding Lugoj...
Expanding Fagaras...
Expanding Mehadia...
Expanding Pitesti...
Expanding Craiova...
Expanding Drobeta...
Expanding Bucharest...
Expanding Urzicenzi...
Expanding Giurgiu...
Expanding Hirsova...
The solution is: ['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest', 'Urzicenzi', 'Vaslui']
The cost is: 645


## 5. Depth-first Search

DFS is often implemented in tree-like search. Recall that in tree-like search, we do not record the expanded nodes.

### *Try it out!*
Copy and modify the function of BFS to implement DFS. A tip: since DFS could get stuck in infinite loops, you can set a limit to the number of iterations. When the number of iterations is reached, terminate the function.

In [20]:
def depth_first_search(graph_to_search, initial_state, goal_state, max_iter=1000, verbose=False):
    # In frontiers, we store a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in the queue represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost of the path.
    frontiers = queue.LifoQueue()
    frontiers.put((0, [initial_state]))
    
    iter_counter = 0

    while not frontiers.empty():   # use while loop to iteratively perform search
        iter_counter += 1
        if max_iter > 0 and iter_counter >= max_iter:
            break

        cost, path = frontiers.get()  # Get the next element in the frontier queue
        node = path[-1]  # Get the last node in this path
        
        if verbose:
            print(f'Expanding {node}...')
        
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_cost = cost + next_cost
            
            # check if we reache the goal state or not
            if next_node == goal_state:
                return new_path, new_cost  # if yes, we can return this path and its cost
            else:
                frontiers.put((new_cost, new_path))  # add to the frontiers

    return None, None

In [21]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using DFS.
# It's ok if it returns None since it could get stuck
path, cost = depth_first_search(romanian_graph, 'Arad', 'Bucharest', max_iter=100, verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad...
Expanding Timisoara...
Expanding Arad..

Let's try it on a single-way version of the Romannian map, which does not have circles.

<div>
    <img src="figs/romanian_graph_single_way.png" width="85%"/>
</div>

In [22]:
single_way_romanian_graph = {
    'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],  # finish the remaining
    'Zerind': [('Oradea', 71)],
    'Oradea': [('Sibiu', 151)],
    'Sibiu': [('Fagaras', 99), ('Rimnicu Vilcea', 80)],
    'Timisoara': [('Lugoj', 111)],
    'Lugoj': [('Mehadia', 70)],
    'Mehadia': [('Drobeta', 75)],
    'Drobeta': [('Craiova', 120)],
    'Rimnicu Vilcea': [('Craiova', 146), ('Pitesti', 97)],
    'Fagaras': [('Bucharest', 211)],
    'Pitesti': [('Bucharest', 101)],
    'Craiova': [('Pitesti', 138)],
    'Bucharest': [('Giurgiu', 90), ('Urzicenzi', 85)],
    'Giurgiu': [],
    'Urzicenzi': [('Vaslui', 142), ('Hirsova', 98)],
    'Vaslui': [('Iasi', 92)],
    'Iasi': [('Neamt', 87)],
    'Neamt': [],
    'Hirsova': [('Eforie', 86)],
    'Eforie': [],
}

In [23]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using DFS.
path, cost = depth_first_search(single_way_romanian_graph, 'Arad', 'Bucharest', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Timisoara...
Expanding Lugoj...
Expanding Mehadia...
Expanding Drobeta...
Expanding Craiova...
Expanding Pitesti...
The solution is: ['Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Craiova', 'Pitesti', 'Bucharest']
The cost is: 733


## 6. Greedy Search
In greedy search, we need to define a **`heuristic function`** $h(n)$ that tells us the estimated cost of the cheapest path from a particular state to a goal state. In this path-finding problem, we make it simpler by using another `dict` to store the straight-line distance from each city to our goal, Bucharest.

In [24]:
def sld_to_bucharest(city):
    """This function returns the straight-line distance from the given city to Bucharest."""
    sld = {
        'Arad': 366,
        'Bucharest': 0,
        'Craiova': 160,
        'Drobeta': 242,
        'Eforie': 161,
        'Fagaras': 176,
        'Giurgiu': 77,
        'Hirsova': 151,
        'Iasi': 226,
        'Lugoj': 244,
        'Mehadia': 241,
        'Neamt': 234,
        'Oradea': 380,
        'Pitesti': 100,
        'Rimnicu Vilcea': 193,
        'Sibiu': 253,
        'Timisoara': 329,
        'Urzicenzi': 80,
        'Vaslui': 199,
        'Zerind': 374
    }
    return sld[city]

### *Try it out!*
The difference between greedy search and BFS is that we expand the node with the *lowest* $h(n)$. Copy the BFS function, rename it to `greedy_search`, and modify it with the help of the heuristic function we defined above to perform the greedy search.

In [25]:
def greedy_search(graph_to_search, initial_state, goal_state, heuristics_function, verbose=False):
    # We add one more argument, heuristics_function, to allow different heuristic functions to be used
    # without changing the implementation source codes.

    frontiers = queue.PriorityQueue()
    frontiers.put((heuristics_function(initial_state), 0, [initial_state])) # a tuple of the cost and the path
    visited = set()  # a set

    while not frontiers.empty():   # use while loop to iteratively perform search
        
        h_value, cost, path = frontiers.get()  # Get the next element in the frontier queue
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then drop it.
            continue
        
        if verbose:
            print(f'Expanding {node}...')
        
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            if next_node in visited:
                continue  # skip this node if it is already expanded.
                
            new_path = path.copy()
            new_path.append(next_node)
            new_cost = cost + next_cost
            
            # check if we reache the goal state or not
            if next_node == goal_state:
                return new_path, new_cost  # if yes, we can return this path and its cost
            else:
                frontiers.put((heuristics_function(next_node), new_cost, new_path))  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited set
        visited.add(node)

    return None, None

In [26]:
# Run this after you implement the algorithm, see if it gives you the correct answer.
# Find a path from Arad to Bucharest in the Romannian map using greedy search

path, cost = greedy_search(romanian_graph, 'Arad', 'Bucharest', heuristics_function=sld_to_bucharest, verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Sibiu...
Expanding Fagaras...
The solution is: ['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
The cost is: 450


## 8. A* Search
For A* search, we compute the sum of the cost of the path $g(n)$ and the heuristic function $h(n)$, and select the node with *lowest* $h(n)+g(n)$ to expand. Besides, similar to the uniform-cost search, we check the goal state when we **expand it**.  Copy the greedy search function, rename it to `a_star_search`, and modify it to perform the A* search. You could make reference to the uniform-cost search codes to check the goal state when expanding the node.

### *Try it out!*
Copy the greedy function, rename it to `a_star_search` and modify it to perform A* search.

In [27]:
def a_star_search(graph_to_search, initial_state, goal_state, heuristics_function, verbose=False):
    # We add one more argument, heuristics_function, to allow different heuristic functions to be used
    # without changing the implementation source codes.

    frontiers = queue.PriorityQueue()
    frontiers.put((heuristics_function(initial_state), 0, [initial_state])) # a tuple of the cost and the path
    visited = set()  # a set

    while not frontiers.empty():   # use while loop to iteratively perform search
        
        h_value, cost, path = frontiers.get()  # Get the next element in the frontier queue
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then drop it.
            continue
            
        if node == goal_state:
            return path, cost
        
        if verbose:
            print(f'Expanding {node}...')
        
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            if next_node in visited:
                continue  # skip this node if it is already expanded.
                
            new_path = path.copy()
            new_path.append(next_node)
            new_cost = cost + next_cost
            
            f_value = heuristics_function(next_node) + next_cost
            frontiers.put((f_value, new_cost, new_path))  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited set
        visited.add(node)

    return None, None

In [28]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using A* search

path, cost = a_star_search(romanian_graph, 'Arad', 'Bucharest', heuristics_function=sld_to_bucharest, verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Sibiu...
Expanding Rimnicu Vilcea...
Expanding Pitesti...
The solution is: ['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']
The cost is: 418


### What happens if we have a heuristic function that overestimates the cost?

### *Try it out!*
Define a heuristic function that returns the doubled straight-line distance from each city to Bucharest. Hint: you do not need to recode the dict again, just make use of the predefined `sld_to_bucharest` function.

In [29]:
def double_sld(x):
    return 2 * sld_to_bucharest(x)

Perform A* search using this new heuristic function to find a way from Arad to Bucharest.

In [30]:
path, cost = a_star_search(romanian_graph, 'Arad', 'Bucharest', heuristics_function=double_sld, verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Expanding Arad...
Expanding Sibiu...
Expanding Fagaras...
The solution is: ['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
The cost is: 450


### Compare the results of using the two different heuristic functions, what are your observations? Try to explain the differences using what we learnt from the lectures.

The SLD heuristic function is admissible, so A* with this heuristic function finds the cost-optimal solution. However, the doubled SLD heuristic function is not because it could overestimate the costs, so A* with this heuristic function fails to find the optimal solution.